In [34]:
from sympy import * 
from spb import *

import numpy as np

import plotly.graph_objects as go
from plotly.subplots import make_subplots

init_printing(pretty_print=True)


## indoor air integral model - volume average indoor air

### Simpler way

In [74]:
class PhysicalConstants:
    rho= 1.225 #kg/m^3 
    cp= 1005 # J/(kg-K) 
    V= 2800 #m^3 
    T0= 293.15 # K 

In [94]:
class EnergyEqns:
    # TODO will need to init by calling the classes that compute these 
    def fabric_at_t(self, t):
        return 5e8*np.sin(t)
    
    def interior_at_t(self,t):
        return 0
    
    def ventilation_at_t(self, t):
        return 0
    
    def energy_at_t(self, t):
        return self.fabric_at_t(t) + self.interior_at_t(t) + self.ventilation_at_t(t)


In [135]:
def calc_transient_indoor_air(pc:PhysicalConstants, e:EnergyEqns, n_mins=60, calcs_per_min=4,):
    # initialize time array 
    times = np.linspace(start=0, stop=n_mins, num=n_mins*calcs_per_min, endpoint=False)
    dt = times[1] - times[0]
    const_exp = dt/(pc.rho*pc.cp*pc.V) 
    
    # initialize temperature array 
    temps = np.zeros(len(times))
    temps[0] = pc.T0

    for ix, _ in enumerate(temps):  
        if ix < len(temps) - 1: 
            temps[ix+1] = temps[ix] + const_exp*e.energy_at_t(times[ix])

    return times, temps

In [136]:
times, temps = calc_transient_indoor_air(PhysicalConstants(), EnergyEqns())
len(temps)

In [137]:
# super quick scatter plot 
fig = go.Figure()
fig.add_trace(go.Scatter(x=times, y=temps, mode='lines+markers',))
fig.update_layout(xaxis_title='Time',
                  yaxis_title='Temperature (ºC)',
                  title='Volume Avg Indoor Air')

In [139]:
# can override fabric equation by extending EnergyEqns class
class EnergyEqns2(EnergyEqns):
    def fabric_at_t(self, t):
        base_n = 1e6
        return base_n*sin(t) if t%3 == 0 else base_n*cos(t)


In [140]:
times, temps = calc_transient_indoor_air(PhysicalConstants(), EnergyEqns2())
len(temps)

# super quick scatter plot 
fig = go.Figure()
fig.add_trace(go.Scatter(x=times, y=temps, mode='lines+markers',))
fig.update_layout(xaxis_title='Time',
                  yaxis_title='Temperature (ºC)',
                  title='Volume Avg Indoor Air - Different Fabric Equation')

### Sympy way 

In [35]:
rho, cp, V, Ef, Eint, Ev, dt, t = symbols("rho_air, cp_air, V_air, E_f, E_int, E_v Delta_t t")

rho, cp, V, Ef, Eint, Ev, dt, t

In [48]:
times = np.linspace(0, 10, 10*4, False) # 1 hr = 60 mins
dt_calc = times[1] - times[0]
times[0:10], dt

(array([0.  , 0.25, 0.5 , 0.75, 1.  , 1.25, 1.5 , 1.75, 2.  , 2.25]), Delta_t)

In [49]:
temp_at_time = {ix: k for ix, k in enumerate(symbols(f"T_int0:{len(times)}"))}
temp_at_time[0]

In [50]:
# approximate E_f as a sinusoidal equation initially 
E_f = sin(t)
E_f

In [51]:
# make system of equations
sys_eqns = []
for k, v in temp_at_time.items():
    if k < len(temp_at_time) - 1:
        sys_eqns.append(Eq(
            temp_at_time[k+1],  
            temp_at_time[k] + (dt/(rho*cp*V) * (E_f.subs(t,k) + Eint + Ev))))

In [52]:
# sub in to create system of equations

subs= {
    rho: 1.225, #kg/m^3 
    cp: 1005, # J/(kg-K) 
    V: 2800, #m^3 
    temp_at_time[0]: 293.15, # K 
    Eint: 0, 
    Ev: 0, 
    dt: dt_calc #s ,
}

In [53]:
eqns_simp = [eqn.subs(subs) for eqn in sys_eqns]
eqns_simp[0:10]

In [42]:
sol = solve(eqns_simp)
sol_vals = np.array([float(v) for v in sol.values()])
sol_vals[0]

In [46]:
# super quick scatter plot 
fig = go.Figure()
fig.add_trace(go.Scatter(x=times, y=sol_vals, mode='lines+markers',))
fig.update_layout(xaxis_title='Minutes',
                  yaxis_title='Temperature (ºC)',
                  title='All Data')